# Resnet 18 from scratch on standard training and attacks on CIFAR 100 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

import warnings
warnings.filterwarnings('ignore')

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [3]:
model_name = "resnet"
num_classes = 100
feature_extract = False

model_scratch, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
train_dataset, val_dataset, train_loader, val_loader = CIFAR100(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"

0it [00:00, ?it/s]

Extracting ./datasets/CIFAR-100/cifar-100-python.tar.gz to ./datasets/CIFAR-100
Files already downloaded and verified


In [21]:
len(train_dataset)

50000

In [4]:
model_scratch = model_scratch.to(device)
params_to_update = model_scratch.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_scratch = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [6]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_scratch_100.pth'
device = "cuda"
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("Standard_Resnet_Scratch_100")

In [7]:
train(model_scratch, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_scratch, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 4.281087502189305	 lr: 0.001
Epoch [1/100]	 Validation Loss: 4.0268026816694045	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 3.9563587657021135	 lr: 0.001
Epoch [2/100]	 Validation Loss: 3.8379239975651607	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 3.8336323545412028	 lr: 0.001
Epoch [3/100]	 Validation Loss: 3.697008836118481	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 3.7369757997410376	 lr: 0.001
Epoch [4/100]	 Validation Loss: 3.6083407734013813	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 3.661471131512576	 lr: 0.001
Epoch [5/100]	 Validation Loss: 3.541160631783401	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 3.6023108288455195	 lr: 0.001
Epoch [6/100]	 Validation Loss: 3.480535781836208	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 3.543496634954077	 lr: 0.001
Epoch [7/100]	 Validation Loss: 3.394083195094821	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 3.4

Epoch [60/100]	 Training Loss: 1.9853497313721407	 lr: 0.001
Epoch [60/100]	 Validation Loss: 1.8492798971224436	 lr: 0.001
Epoch [60/100]	
Epoch [61/100]	 Training Loss: 1.967300393697246	 lr: 0.001
Epoch [61/100]	 Validation Loss: 1.8423214532152008	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 1.9443874020710625	 lr: 0.001
Epoch [62/100]	 Validation Loss: 1.8205428621436976	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 1.928411097477769	 lr: 0.001
Epoch [63/100]	 Validation Loss: 1.7704804728302774	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.9143045692492628	 lr: 0.001
Epoch [64/100]	 Validation Loss: 1.785844952245302	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.9120185808147616	 lr: 0.001
Epoch [65/100]	 Validation Loss: 1.7999907490573352	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.8831863141120853	 lr: 0.001
Epoch [66/100]	 Validation Loss: 1.7379785338534584	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Trainin

In [8]:
print("the clean accuracy is ")
test(val_loader, model_scratch)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 58 %


In [9]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_scratch, "cuda")

PGD
[Model loaded]
Acc: 1.72 %
adv model


In [10]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_scratch, "cuda")

fgsm
[Model loaded]
Acc: 9.63 %
adv model


In [11]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_scratch, "cuda")

nifgsm
[Model loaded]
Acc: 3.58 %
adv model


In [12]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_scratch, "cuda")

vmifgsm
[Model loaded]
Acc: 1.90 %
adv model


# ResNet 18 from scratch on adversarial training and attacks on CIFAR 100

In [13]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_scratch_adv = copy.deepcopy(model_scratch) #am I doing correct? 
# for param in net_adv.parameters():
#     print(param)

In [14]:
params_to_update = net_scratch_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_scratch_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss() 
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_scratch_adv_pgd.pth' #to do change to 100 
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_scratch_PGD")
pgd_attack = generate_adv(model_scratch, "pgd")

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [15]:
train(net_scratch_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 3.1038518092211556	 lr: 0.001
Epoch [1/100]	 Validation Loss: 31.664396117005168	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.7384813709941973	 lr: 0.001
Epoch [2/100]	 Validation Loss: 48.088782877861696	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.6230291643410997	 lr: 0.001
Epoch [3/100]	 Validation Loss: 48.43528839304477	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.544090086846705	 lr: 0.001
Epoch [4/100]	 Validation Loss: 48.8614333430423	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.494838892346453	 lr: 0.001
Epoch [5/100]	 Validation Loss: 51.19081535822229	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.449356273616976	 lr: 0.001
Epoch [6/100]	 Validation Loss: 51.079985316795636	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 2.4091631661900474	 lr: 0.001
Epoch [7/100]	 Validation Loss: 49.08472104615803	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 2.37

Epoch [61/100]	 Training Loss: 1.2372887020220842	 lr: 0.001
Epoch [61/100]	 Validation Loss: 67.0816765797289	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 1.2389980845744042	 lr: 0.001
Epoch [62/100]	 Validation Loss: 67.47748184204102	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 1.2130697229329277	 lr: 0.001
Epoch [63/100]	 Validation Loss: 58.50753576544267	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.1956222848514158	 lr: 0.001
Epoch [64/100]	 Validation Loss: 58.214483236964746	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.1887955351558792	 lr: 0.001
Epoch [65/100]	 Validation Loss: 66.11614975748184	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.1785911507618703	 lr: 0.001
Epoch [66/100]	 Validation Loss: 60.08599742454818	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 1.176644406202809	 lr: 0.001
Epoch [67/100]	 Validation Loss: 60.74316782890996	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training Los

In [16]:
print("the clean accuracy is ")
test(val_loader, net_scratch_adv)

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110
the clean accuracy is 
Accuracy of the network on the 10000 test images: 1 %


In [17]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_scratch_adv, "cuda")

PGD
[Model loaded]
Acc: 64.19 %
adv model


In [18]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_scratch_adv, "cuda")

fgsm
[Model loaded]
Acc: 58.32 %
adv model


In [19]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_scratch_adv, "cuda")

nifgsm
[Model loaded]
Acc: 67.70 %
adv model


In [20]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_scratch_adv, "cuda")

vmifgsm
[Model loaded]
Acc: 70.20 %
adv model
